# komet : Kronecker Optimized METhod for DTI prediction

1. Downloading the dataset (train/val/test) : you can choose different databases 
2. Choosing parameters of approximation (Nystrom and features dimension)
3. Calculating of molecule features using a subsample of train molecules (MorganFP kernel approximated via Nystrom approximation)
4. Loading approximated protein features, using SVD of the Local Alignment kernel precalculated on 20605 human proteins
5. Searching for the best lambda by choosing the best AUPR on the validation dataset
6. Testing with best lambda

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zipfile
import pickle

import time 

from sklearn.metrics import  average_precision_score,  roc_curve, confusion_matrix, precision_score, recall_score, auc

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")
device_cpu = device
print( device )

mytype = torch.float16 # to save memory (only on GPU)
mytype = torch.float32

In [ ]:
!pip install rdkit

## 1. Download from a GitHub repo.

### Download utils

In [ ]:
!wget -q https://github.com/Guichaoua/komet/raw/main/komet/komet.py

In [ ]:
import komet

### Download train/val/test
Exemples of dataset : 
* LCIdb/Orphan, 
* BIOSNAP(full_data,unseen_drug, unseen_protein), 
* BindingDB. 
  
These last datasets can be also downloaded on MolTrans Github ex: https://raw.githubusercontent.com/kexinhuang12345/MolTrans/master/dataset/DAVIS/test.csv)

In [ ]:
!mkdir data/
!wget -q https://github.com/Guichaoua/komet/raw/main/data/LCIdb/Orphan/test.csv
!mv test.csv data/
!wget -q https://github.com/Guichaoua/komet/raw/main/data/LCIdb/Orphan/train.csv.zip
!mv train.csv.zip data/
!wget -q https://github.com/Guichaoua/komet/raw/main/data/LCIdb/Orphan/val.csv
!mv val.csv data/
!wget -q https://github.com/Guichaoua/komet/raw/main/data/dict_ind2fasta_all.data
!mv dict_ind2fasta_all.data data/
!wget -q https://github.com/Guichaoua/komet/raw/main/data/U_small.npy
!mv U_small.npy data/
!wget -q https://github.com/Guichaoua/komet/raw/main/data/Lambda_small.npy
!mv Lambda_small.npy data/

In [ ]:
dataset_dir = "data/"

# load data
train = komet.load_df("train.csv.zip",dataset_dir)
val = komet.load_df("val.csv",dataset_dir)
test = komet.load_df("test.csv",dataset_dir)

# dataframe full has all smiles and fasta sequences
full = pd.concat([train, val, test])
print("full shape",full.shape)

## 2. Choose parameters of approximation 

In [ ]:
mM = 3000  #number of molecules to compute molecular features
dM =  1000  #dimension of molecular features
dP = 1200  # number of proteins to compute protein features (rP <=1200 because of the size of U_small.npy)

## 3. Calculation of molecule features using a subsample of train molecules (molecule kernel approximated via Nystrom approximation)

In [ ]:
#### MOLECULE####
# Index of the smiles in the dataset
list_smiles = full[['SMILES']].drop_duplicates().values.flatten()
nM = len(list_smiles)
print("number of different smiles (mol):",nM)
dict_smiles2ind = {list_smiles[i]:i for i in range(nM)}

# add indsmiles in train, val, test
train['indsmiles'] = train['SMILES'].apply(lambda x:dict_smiles2ind[x] )
val['indsmiles'] = val['SMILES'].apply(lambda x: dict_smiles2ind[x])
test['indsmiles'] = test['SMILES'].apply(lambda x: dict_smiles2ind[x])

In [ ]:
# compute the Nystrom approximation of the mol kernel and the features of the Kronecker kernel
nM = len(smiles)
rM = min(mM,nM) # number of molecule to compute nystrom
dM = min(dM,nM) # final dimension of features for molecules
print("rM",rM,"dM",dM)

In [ ]:
# molecule kernel_first step : computation of Morgan fingerprint for all molecules
MorganFP = Morgan_FP(smiles)
S = np.random.permutation(nM)[:rM]
S = np.sort(S)
K_S = ( MorganFP[S,:] @ MorganFP.T ) / ( 1024 - (1-MorganFP[S,:]) @ (1-MorganFP.T) )
print("K_S shape",K_S.shape)
K_SS = K_S[:,S]
print("K_SS shape",K_SS.shape)
# compute the approximate mol features with SVD
V, Mu, VT = torch.svd(K_SS)
#compute the molecule features
epsi = 1e-8  # be careful when we divide by Mu near 0
X = K_S.T @ V[:,:dM] @ torch.diag(1./torch.sqrt(epsi + Mu[:dM]))
print("mol features shape",X.shape)

In [ ]:
# plot the cumulative sum of the eigenvalues to choose the dimension of the features
from matplotlib import pyplot as plt
Mu_c = (Mu**2).cpu().numpy()
plt.plot(np.cumsum(Mu_c )/Mu_c.sum()) # plot the cumulative sum of the eigenvalues
# plot an horizontal line y = 0.99
plt.plot([0,len(Mu)],[0.99,0.99], 'r--')
plt.show()

## 4. Approximated protein features, using SVD of the Local Alignment kernel precalculated on 20605 human proteins 

$K_P = U \Lambda U.T$  

With file size constraints on Github, we can only load $U_{small} = U[:,:1200]$ and $\Lambda_{small} = \Lambda[:,:1200]$

You can look in dict_ind2fasta_all.data which proteins are present.

In [ ]:
# Load Precomputed kernel and dict for the proteins
dict_ind2fasta_all = pickle.load(open("data/dict_ind2fasta_all.data", 'rb'))
dict_fasta2ind_all = {fasta:ind for ind,fasta in dict_ind2fasta_all.items()}
U = torch.from_numpy(np.load("data/U_small.npy"))
Lambda = torch.from_numpy(np.load("data/Lambda_small.npy"))
print("U.shape",U.shape,"Lambda.shape",Lambda.shape)

In [ ]:
from matplotlib import pyplot as plt
Lambda_c = Lambda**2
plt.plot(np.cumsum(Lambda_c )/Lambda_c.sum()) # 99% of the energy is in the first 5000 features
# plot an horizontal line y = 0.95
plt.plot([0,len(Lambda)],[0.95,0.95], 'r--')
plt.show()

In [ ]:
# computation of feature for protein (no nystrom, just SVD)
dP = min(U.shape[0],1200)
Y_all = U[:,:dP] @ torch.diag(torch.sqrt(Lambda[:dP]))
Y_all = U[:,:dP] @ torch.diag(torch.sqrt(Lambda[:dP]))
Y_all = Y_all.to(device)

In [ ]:
# Index of the protein in the dataset
train, fasta = add_indfasta(train)
nP = len(fasta)

In [ ]:
I_fasta = [dict_fasta2ind_all[fasta[i]] for i in range(len(fasta))] # index of fasta in the precomputed dict and protein kernel, in the same order as the dataset
Y = Y_all[I_fasta,:]
print("features shape",Y.shape)

### INDEX OF INTERACTIONS

In [ ]:
# TRAIN
I, J, y = load_datas(train)
n = len(I)
print("len(train)",n)

## Komet algorithm


### training with a choosen lambda

In [ ]:
lamb = 0.01
# train the model
w,b,history_lbfgs_SVM = SVM_bfgs(X,Y,y,I,J,lamb,niter = 50)
# compute a probability using weights (Platt scaling)
s,t,history_lbfgs_Platt = compute_proba_Platt_Scalling(w,X,Y,y,I,J)

### validation 

In [ ]:
# load data
val = load_df("val.csv")

In [ ]:
#### MOLECULE####
# add index and smiles to val
val,smiles_val =  add_indsmiles(val)

# molecule kernel_first step : computation of Morgan fingerprint
MorganFP_val= Morgan_FP(smiles_val)

# compute the Nystrom approximation of the mol kernel and the features of the Kronecker kernel
K_S_val = ( MorganFP[S,:] @ MorganFP_val.T ) / ( 1024 - (1-MorganFP[S,:]) @ (1-MorganFP_val.T) )
print("K_S_val shape",K_S_val.shape)

X_val = K_S_val.T @ V[:,:dM] @ torch.diag(1./torch.sqrt(epsi + Mu[:dM]))
print("mol features val shape",X_val.shape)

In [ ]:
#### protein features for validation set
# Index of the protein in the dataset
val, fasta_val = add_indfasta(val)

I_fasta_val = [dict_fasta2ind_all[fasta_val[i]] for i in range(len(fasta_val))] # index of fasta in the precomputed dict and protein kernel, in the same order as the dataset

Y_val = Y_all[I_fasta_val,:]
print("prot features val shape",Y_val.shape)

In [ ]:
# val
#VALIDATION 
I_val, J_val, y_val = load_datas(val)
n_val = len(I_val)
print("len(val)",n_val)

In [ ]:
# accuracy on the validation set
m,y_pred_val, proba_pred_val = compute_proba(w,b,s,t,X_val,Y_val,I_val,J_val)
# we compute the results
acc1,au_Roc,au_PR,thred_optim,acc_best,cm,FP = results(y_val.cpu(),y_pred_val.cpu(),proba_pred_val.cpu())
print("roc AUC = ",au_Roc)
print("AUPR = ",au_PR)

## Choice of $\lambda$:  Search for the best lambda by choosing the best AUPR on the validation dataset

In [ ]:
#### Choice of the hyperparameters ####
# we use the validation set to choose the hyperparameters
# we use the AUPR as a criterion
lambdas = np.logspace(-11, 2, num=14)
accs = []
aupr = []
lambda_max = 0
aupr_max = 0

for lamb in lambdas:
    print("lambda=",lamb)

    # train the model
    w,b,h = SVM_bfgs(X,Y,y,I,J,lamb)
    # compute a probability using weights (Platt scaling)
    s,t,h = compute_proba_Platt_Scalling(w,X,Y,y,I,J)

    # accuracy on the validation set
    m,y_pred_val, proba_pred_val = compute_proba(w,b,s,t,X_val,Y_val,I_val,J_val)
    # we compute the results
    acc1,au_Roc,au_PR,thred_optim,acc_best,cm,FP = results(y_val.cpu(),y_pred_val.cpu(),proba_pred_val.cpu())

    accs.append(acc1)
    aupr.append(au_PR)

    if au_PR > aupr_max:
        lambda_max=lamb
        aupr_max = au_PR

print("-"*30)
print("lambda_max",lambda_max)
print("aupr_max for val",aupr_max)


Plot accuracy in function of lambda

In [ ]:
import matplotlib.pyplot as plt
plt.plot(lambdas,accs)
plt.xscale('log')
plt.title('Accuracy in function of lambda')
plt.show()

In [ ]:
plt.plot(lambdas,aupr)
plt.xscale('log')
plt.title('AUPR in function of lambda')
plt.show()

## Training with lambda_max

In [ ]:
lamb = lambda_max
print("lambda_max=",lamb)
# train the model
w_bfgs,b_bfgs = SVM_bfgs(X,Y,y,I,J,lamb)
# compute a probability using weights (Platt scaling)
s,t = compute_proba_Platt_Scalling(w_bfgs,X,Y,y,I,J)

## Test

In [ ]:
# load data
test = load_df("test.csv")

In [ ]:
#### MOLECULE features for test set
# Index of the smiles in the dataset
test,smiles_test =  add_indsmiles(test)

X_test = Nystrom_X(smiles_test,S,MorganFP,V,dM,Mu,epsi)
print("mol features test shape",X_test.shape)

In [ ]:
#### protein features for test set
# Index of the protein in the dataset
test,fasta_test = add_indfasta(test)

I_fasta_test = [dict_fasta2ind_all[fasta_test[i]] for i in range(len(fasta_test))] # index of fasta in the precomputed dict and protein kernel, in the same order as the dataset

Y_test = Y_all[I_fasta_test,:]
print("prot features test shape",Y_test.shape)

In [ ]:
# test
I_test, J_test, y_test = load_datas(test)
n_test = len(I_test)
print("len(test)",n_test)

In [ ]:
# we compute a probability using weights (Platt scaling)
m,y_pred, proba_pred = compute_proba(w_bfgs,b_bfgs,s,t,X_test,Y_test,I_test,J_test)
# we compute the results
acc1,au_Roc,au_PR,thred_optim,acc_best,cm,FP = results(y_test.cpu(),y_pred.cpu(),proba_pred.cpu())
print(f"roc AUC = {au_Roc:.4f}")
print(f"AUPR = {au_PR:.4f}")
print(f"accuracy (threshold 0.5)= {acc1:.4f}")
print(f"best threshold = {thred_optim:.4f}")
print(f"accuracy (best threshold)= {acc_best:.4f}")
print(f"false positive (best threshold)= {FP:.4f}")

In [ ]:
# plot confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay
labels = [-1., 1.]
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=labels)
disp.plot()
plt.show()

In [ ]:
# plot distribution (density) of p when y_test=1
plt.hist(proba_pred.cpu().numpy()[y_test.cpu().numpy()==1],bins=10,alpha=0.8,color='green',label='y_test=1');
plt.hist(proba_pred.cpu().numpy()[y_test.cpu()==-1],bins=10,alpha=0.5,color='red',label='y_test=-1');
plt.legend()
plt.title('Distribution of predicted probability')
plt.show()